### En este notebook vamos a explicar el proceso de data cleaning y a responder las preguntas en python propuestas

In [ ]:
#Vamos a realizar un modelo de classification en los datos de unos clientes de banco 
#para estudiar y predecir el comportamiento de los clientes en función de sus características

#primero importamos las librerías necesarias
import pandas as pd
import os
pd.set_option('display.max_columns', None)
import numpy as np
import streamlit as st

import pylab as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, confusion_matrix, recall_score, precision_score

from imblearn.over_sampling import SMOTE, RandomOverSampler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importamos los datos
data = pd.read_csv('./data/creditcardmarketing.csv')

1. 'Select all the data from table `credit_card_data` to check if the data was imported correctly.'

In [ ]:
print(data) #vemos que se ha importado correctamente

2. 'Select all the data from the table to verify if the command worked. Limit your returned results to 10.'

In [ ]:
print(data.head(10), "\n") #vemos las 10 primeras filas del dataset

3. 'Use code to find how many rows of data you have.'

In [ ]:
#sacamos la información de los datos para revisar si necesitamos hacer algún ajuste a nuestros dataset
#analizamos el dataset

print("Estructura de los datos: ", data.shape, "\n") #vemos las filas y columnas que tenemos
print(data.describe(), "\n")
print(data.info(), "\n")
print(data.isnull().sum(), "\n")
#Inside de lo printeado

#Hay varias columnas con varos nulos que analizaremos más adelante, y varias columnas no numericas
#que también modificaremos.

In [ ]:
#vamos con los valores nulos. Como son numericos los rellenaremos con la media de la columna
#las columnas que vamos a rellenar con la media son las siguientes: average_balance,balanceq1,balanceq2,balanceq3,balanceq4

data['average_balance'].fillna(data['average_balance'].mean(), inplace=True)
data['balanceq1'].fillna(data['balanceq1'].mean(), inplace=True)
data['balanceq2'].fillna(data['balanceq2'].mean(), inplace=True)
data['balanceq3'].fillna(data['balanceq3'].mean(), inplace=True)
data['balanceq4'].fillna(data['balanceq4'].mean(), inplace=True)


#comprobamos que no hay más valores nulos
print(data.isnull().sum(), "\n")
#no los hay, asi que continuamos

4. 'Now find the unique values in some of columns:

    - What are the unique values in the column `Offer Accepted`?
    - What are the unique values in the column `Reward`?
    - What are the unique values in the column `Mailer Type`?
    - What are the unique values in the column `# Credit Cards Held`?
    - What are the unique values in the column `household_size`?'


In [ ]:

#ahora vamos a gestionar las columnas no numéricas
#vamos a ver los valores unicos de estas columnas: offer_acepted,reward,mailer_type,credits_cards_held,household_size

print(data['offer_acepted'].unique(), "\n")
print(data['reward'].unique(), "\n")
print(data['mailer_type'].unique(), "\n")
print(data['credits_cards_held'].unique(), "\n")
print(data['household_size'].unique(), "\n")

#vamos cambiar a numericos los valores de las columnas offer_acepted,reward, mailer_type, overdraft_protection, own_home

data['offer_acepted'] = data['offer_acepted'].map({'No': 0, 'Yes': 1})
data['reward'] = data['reward'].map({'Air Miles': 0, 'Cash Back': 1, 'Points': 2})
data['mailer_type'] = data['mailer_type'].map({'Letter': 0, 'Postcard': 1})
data['overdraft_protection'] = data['overdraft_protection'].map({'No': 0, 'Yes': 1})
data['own_home'] = data['own_home'].map({'No': 0, 'Yes': 1})


#comprobamos que ha funcionado printeando los valores unicos de nuevo
print(data['offer_acepted'].unique(), "\n")
print(data['reward'].unique(), "\n")
print(data['mailer_type'].unique(), "\n")
print(data['overdraft_protection'].unique(), "\n")
print(data['own_home'].unique(), "\n")

#todo perfect(las demás columnas no numéricas las vamos a ver más adelante)

5. 'Arrange the data in decreasing order by the `average_balance`. Return only the `customer_number` of the top 10 customers with the highest `average_balances` in your data.'

In [ ]:
#vamos a sacar en orden descendente los 10 clientes con mayor balance medio con su balance medio
data1 = data.sort_values(by='average_balance', ascending=False)
print(data1[['customer_number', 'average_balance']].head(10), "\n")

6.  'What is the average of `Average Balance` of all the customers in your data?'

In [ ]:
#vamos a sacar la media de la columna average_balance, es decir, la media del balance medio de los clientes
print(data['average_balance'].mean(), "\n")

7. 'In this exercise use  `groupby` to check the properties of some of the categorical variables in our data. Note wherever `average_balance` is asked in the questions below, please take the average of the column `average_balance`. Show the results in a data frame and a plot. 

    - What is the average balance of the customers grouped by `Income Level`? The returned result should have only two columns, `Income` and `Average Balance` of the customers. 
    - What is the average balance of the customers grouped by `number_of_bank_accounts_open`? The returned result should have only two columns, `number_of_bank_aaccounts_open` and `Average Balance` of the customers. 
    - What is the average number of credit cards held by customers for each of the credit card ratings? The returned result should have only two columns, `rating` and `average number of credit cards`.
    - Is there any correlation between the columns `credit_cards_held` and `number_of_bank_accounts_open`? You can analyze this by grouping the data by one of the variables and then aggregating the results of the other column. Visually check if there is a positive correlation or negative correlation or no correlation between the variables.
    - Check the number of customers in each category (ie number of credit cards held) to assess if that category is well represented in the dataset to include it in your analysis. For eg. If the category is under-represented as compared to other categories, ignore that category in this analysis
'

In [ ]:

#vamos a sacar la media del balance medio de los clientes agrupados por nivel de ingresos
print(data.groupby('income_level')[['income_level', 'average_balance']].mean(), "\n")

#vamos a sacar la media del balance medio de los clientes agrupados por número de cuentas bancarias abiertas
print(data.groupby('banks_accounts_open')['average_balance'].mean(), "\n")

#vamos a sacar la media del número de tarjetas de crédito que tienen los clientes agrupados por rating de tarjetas de crédito
print(data.groupby('credit_rating')[['credit_rating', 'credits_cards_held']].mean(), "\n")

#vamos a sacar la media del número de tarjetas de crédito que tienen los clientes agrupados por número de cuentas bancarias abiertas
print(data.groupby('banks_accounts_open')[['banks_accounts_open', 'credits_cards_held']].mean(), "\n")
#estos datos nos aproximan la correlacion entre el numero de cuentas bancarias abiertas y el numero de tarjetas de credito que tienen los clientes,
#pero para conocer con más precisión el resultado podemos fijarnos en la matriz de correlacion del principio
#conclusión: no hay correlacion significativa entre el numero de cuentas bancarias abiertas y el numero de tarjetas de credito que tienen los clientes

#vamos a sacar el número de clientes en cada categoría (es decir, número de tarjetas de crédito que tienen) para evaluar si esa categoría está bien representada en el conjunto de datos para incluirla en su análisis. Por ejemplo. Si la categoría está subrepresentada en comparación con otras categorías, ignoraremos esa categoría en este análisis
print(data.groupby('credits_cards_held')['customer_number'].count(), "\n")


8. 'Your managers are only interested in the customers with the following properties:

    - Credit rating medium or high
    - Credit cards held 2 or less
    - Owns their own home
    - Household size 3 or more

    For the rest of the things, they are not too concerned. Write code to find what are the options available for them. 

    Can you filter the customers who accepted the offers here?
'

In [ ]:

#ahora vamos a filtrar y sacar los clientes que tienen credit_rating = medium o high, credit_cards_held = 2 o menos, own_home = Yes y household_size = 3 o más
data_interesante = data[(data['credit_rating'] == 'Medium') | (data['credit_rating'] == 'High') & (data['credits_cards_held'] <= 2) & (data['own_home'] == 'Yes') & (data['household_size'] >= 3)]
print(data_interesante, "\n")

9. 'Your managers want to find out the list of customers whose average balance is less than the average balance of all the customers in the database. Write a query to show them the list of such customers.'

In [ ]:

#sacamos los clientes con un average_balance < a la media de la columna average_balance
data_average_balance = data[data['average_balance'] < data['average_balance'].mean()]
data_average_balance.sort_values(by='average_balance', ascending=True)
print(data_average_balance[['customer_number', 'average_balance']], "\n")

10. 'What is the number of people who accepted the offer vs the number of people who did not?'

In [ ]:


#sacamos el numero de clientes que aceptan la oferta y el numero de clientes que no aceptan la oferta
print(data.groupby('offer_acepted')['customer_number'].count(), "\n")
#recordar 0 = oferta no aceptada y 1 = oferta aceptada

11. 'Your managers are more interested in customers with a credit rating of high or medium. What is the difference in average balances of the customers with high credit card rating and low credit card rating?'

In [ ]:


#sacamos los datos con credit rating = high or low
data_credit_rating = data[(data['credit_rating'] == 'High') | (data['credit_rating'] == 'Low')]
#sacamos la media de el average balance de los clientes con credit rating = high or low
print(data_credit_rating.groupby('credit_rating')[['credit_rating', 'average_balance']].mean(), "\n")
print("La diferencia entre la media de balances de aquellos clientes con un rating alto y uno bajo es ", (data_credit_rating.groupby('credit_rating')['average_balance'].mean().iloc[0]-data_credit_rating.groupby('credit_rating')['average_balance'].mean().iloc[1]), "\n")

12. 'In the database, which all types of communication (`Mailer Type`) were used and with how many customers?'

In [ ]:

#vamos a ver los tipos de mailer_type que hay y con cuantos clientes se han usado
print(data.groupby('mailer_type')['customer_number'].count(), "\n")

13. 'Provide the details of the customer that is the 11th least `Q1_balance` in your database.'

In [ ]:
#por ultimo vamos a sacar al onceavo cliente con menos balanceq1 en nuestro dataset
data2 = data.sort_values(by='balanceq1', ascending=True)
print(data2.iloc[10], "\n")

Terminamos después de las preguntas el data cleaning

In [ ]:


#vamos ahora a terminar con las columnas no numéricas
print(data.dtypes, "\n")
#las columnas balanceq1, balanceq2, balanceq3 y balanceq4 no nos sirven para el modelo de clasificacion asi que las vamos a eliminar
data = data.drop(['balanceq1', 'balanceq2', 'balanceq3', 'balanceq4'], axis=1)
#ahora con las dos columnas que nos quedan vamos a cambiarlas a numericas manualmente
data['credit_rating'] = data['credit_rating'].map({'Low': 0, 'Medium': 1, 'High': 2})
data['income_level'] = data['income_level'].map({'Low': 0, 'Medium': 1, 'High': 2})
#volvemos a ver los tipos de datos
print(data.dtypes, "\n")
#ya tenemos todas las columnas numericas
#finalmente vamos a guardar el dataset en un nuevo csv
data.to_csv('data/creditcardmarketing_clean.csv', index=False)

#con el dataset ya limpio, volvemos a ver la matriz de correlacion
plt.figure(figsize=(15, 10))
sns.set(style='white')
mask=np.triu(np.ones_like(data.corr(), dtype=bool))
cmap=sns.diverging_palette(0, 10, as_cmap=True)
sns.heatmap(data.corr(),
          mask=mask,
          cmap=cmap,
          center=0,
          square=True,
          annot=True,
          linewidths=0.5,
          cbar_kws={'shrink': 0.5})
plt.show()